# Simulação de Distribuição Binomial: Experimentação e Visualização

Este notebook apresenta simulações práticas da distribuição binomial, permitindo que você experimente e visualize os conceitos em ação. Utilizaremos o padrão BDD (Behavior Driven Development) para estruturar nossos cenários de forma clara e didática.

## 📚 Recordando: O que é a Distribuição Binomial?

A distribuição binomial modela situações onde:
- Realizamos **n** experimentos independentes
- Cada experimento tem apenas dois resultados possíveis: **sucesso** ou **fracasso**
- A probabilidade de sucesso **p** é constante em todas as tentativas
- Queremos saber a probabilidade de obter exatamente **k** sucessos

**Fórmula:** P(X = k) = C(n,k) × p^k × (1-p)^(n-k)

In [ ]:
# Importação das bibliotecas necessárias
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import binom
from scipy.special import comb
import pandas as pd
from IPython.display import display, Markdown

# Configuração dos gráficos
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✅ Bibliotecas importadas com sucesso!")
print("Versões utilizadas:")
print(f"NumPy: {np.__version__}")
print(f"Matplotlib: {plt.matplotlib.__version__}")
print(f"Pandas: {pd.__version__}")

---
## 🎯 Cenário 1: Prova de Múltipla Escolha

**DADO** que um estudante fará uma prova com 5 questões de múltipla escolha  
**E** cada questão tem 4 alternativas (probabilidade de acerto por chute = 0.25)  
**QUANDO** o estudante responde todas as questões chutando  
**ENTÃO** queremos analisar as probabilidades de diferentes números de acertos

In [ ]:
# DADO: Parâmetros do cenário
n_questoes = 5
p_acerto_chute = 0.25  # 1 em 4 alternativas
nome_cenario = "Prova de Múltipla Escolha"

print(f"📊 {nome_cenario}")
print(f"Número de questões (n): {n_questoes}")
print(f"Probabilidade de acerto por chute (p): {p_acerto_chute}")
print(f"Probabilidade de erro (1-p): {1-p_acerto_chute}")

# E: Definindo o espaço amostral
k_valores = np.arange(0, n_questoes + 1)
print(f"\nPossíveis números de acertos: {list(k_valores)}")

In [ ]:
# QUANDO: Calculamos as probabilidades para cada número de acertos
probabilidades = []
detalhes_calculo = []

print("🧮 Calculando probabilidades passo a passo:\n")

for k in k_valores:
    # Cálculo manual usando a fórmula
    coef_binomial = comb(n_questoes, k, exact=True)
    prob_sucessos = p_acerto_chute ** k
    prob_fracassos = (1 - p_acerto_chute) ** (n_questoes - k)
    prob_final = coef_binomial * prob_sucessos * prob_fracassos
    
    # Usando scipy para verificação
    prob_scipy = binom.pmf(k, n_questoes, p_acerto_chute)
    
    probabilidades.append(prob_final)
    
    print(f"P(X = {k}) = C({n_questoes},{k}) × {p_acerto_chute}^{k} × {1-p_acerto_chute}^{n_questoes-k}")
    print(f"       = {coef_binomial} × {prob_sucessos:.6f} × {prob_fracassos:.6f}")
    print(f"       = {prob_final:.6f} ({prob_final*100:.2f}%)")
    print(f"Verificação (scipy): {prob_scipy:.6f}\n")
    
    detalhes_calculo.append({
        'k': k,
        'C(n,k)': coef_binomial,
        'p^k': prob_sucessos,
        '(1-p)^(n-k)': prob_fracassos,
        'Probabilidade': prob_final,
        'Percentual': prob_final * 100
    })

# Verificação: soma deve ser 1
soma_prob = sum(probabilidades)
print(f"✅ Verificação: Soma das probabilidades = {soma_prob:.10f}")

In [ ]:
# ENTÃO: Visualizando e analisando os resultados
df_resultado = pd.DataFrame(detalhes_calculo)

print("📋 Tabela de Resultados:")
display(df_resultado.round(6))

# Criando visualizações
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Gráfico 1: Distribuição de probabilidades
bars = ax1.bar(k_valores, probabilidades, color='skyblue', alpha=0.8, edgecolor='navy')
ax1.set_title(f'{nome_cenario}\nDistribuição de Probabilidades', fontsize=12, fontweight='bold')
ax1.set_xlabel('Número de acertos (k)')
ax1.set_ylabel('Probabilidade P(X = k)')
ax1.grid(True, alpha=0.3)

# Adicionando valores nas barras
for bar, prob in zip(bars, probabilidades):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{prob:.3f}\n({prob*100:.1f}%)', 
             ha='center', va='bottom', fontsize=9)

# Gráfico 2: Probabilidade acumulada
prob_acumulada = np.cumsum(probabilidades)
ax2.step(k_valores, prob_acumulada, where='mid', marker='o', color='red', linewidth=2)
ax2.fill_between(k_valores, prob_acumulada, step='mid', alpha=0.3, color='red')
ax2.set_title('Distribuição Acumulada', fontsize=12, fontweight='bold')
ax2.set_xlabel('Número de acertos (k)')
ax2.set_ylabel('P(X ≤ k)')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 1.05)

# Gráfico 3: Comparação com distribuição normal
media = n_questoes * p_acerto_chute
variancia = n_questoes * p_acerto_chute * (1 - p_acerto_chute)
desvio_padrao = np.sqrt(variancia)

x_continuo = np.linspace(-0.5, n_questoes + 0.5, 1000)
y_normal = (1/np.sqrt(2*np.pi*variancia)) * np.exp(-0.5*((x_continuo - media)**2)/variancia)

ax3.bar(k_valores, probabilidades, color='lightblue', alpha=0.7, label='Binomial')
ax3.plot(x_continuo, y_normal, 'r-', linewidth=2, label=f'Normal(μ={media:.2f}, σ={desvio_padrao:.2f})')
ax3.set_title('Comparação: Binomial vs Normal', fontsize=12, fontweight='bold')
ax3.set_xlabel('Número de acertos (k)')
ax3.set_ylabel('Densidade/Probabilidade')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Gráfico 4: Análise das propriedades
propriedades = {
    'Média (μ)': media,
    'Variância (σ²)': variancia,
    'Desvio Padrão (σ)': desvio_padrao,
    'Moda': k_valores[np.argmax(probabilidades)],
    'P(X ≥ 3)': sum(probabilidades[3:]),
    'P(X ≤ 1)': sum(probabilidades[:2])
}

props_nomes = list(propriedades.keys())
props_valores = list(propriedades.values())

colors = plt.cm.Set3(np.linspace(0, 1, len(props_nomes)))
bars = ax4.barh(props_nomes, props_valores, color=colors)
ax4.set_title('Propriedades da Distribuição', fontsize=12, fontweight='bold')
ax4.set_xlabel('Valores')

# Adicionando valores nas barras
for bar, valor in zip(bars, props_valores):
    ax4.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, 
             f'{valor:.3f}', ha='left', va='center', fontsize=9)

plt.tight_layout()
plt.show()

print(f"\n📊 Análise dos Resultados:")
print(f"• Média de acertos esperada: {media:.2f}")
print(f"• Desvio padrão: {desvio_padrao:.2f}")
print(f"• Número mais provável de acertos: {k_valores[np.argmax(probabilidades)]}")
print(f"• Probabilidade de passar (≥3 acertos): {sum(probabilidades[3:])*100:.2f}%")
print(f"• Probabilidade de ter ≤1 acerto: {sum(probabilidades[:2])*100:.2f}%")

---
## 🏭 Cenário 2: Controle de Qualidade em Produção

**DADO** que uma fábrica produz peças com 10% de probabilidade de defeito  
**E** selecionamos uma amostra de 8 peças aleatoriamente  
**QUANDO** analisamos essa amostra  
**ENTÃO** queremos determinar as probabilidades de encontrar diferentes quantidades de peças defeituosas

In [ ]:
# DADO: Parâmetros do cenário de controle de qualidade
n_pecas = 8
p_defeito = 0.1
nome_cenario_2 = "Controle de Qualidade"

print(f"🏭 {nome_cenario_2}")
print(f"Tamanho da amostra (n): {n_pecas}")
print(f"Probabilidade de defeito (p): {p_defeito}")
print(f"Probabilidade de peça boa (1-p): {1-p_defeito}")

# E: Simulação de múltiplas amostras
print(f"\n🎲 Simulando {1000} amostras de {n_pecas} peças cada...")
np.random.seed(42)  # Para reprodutibilidade
simulacoes = np.random.binomial(n_pecas, p_defeito, 1000)
print(f"Simulações realizadas: {len(simulacoes)} amostras")

In [ ]:
# QUANDO: Calculamos probabilidades teóricas e comparamos com simulação
k_valores_2 = np.arange(0, n_pecas + 1)
prob_teoricas = binom.pmf(k_valores_2, n_pecas, p_defeito)

# Calculando frequências da simulação
freq_simulacao = np.bincount(simulacoes, minlength=n_pecas+1)
prob_simulacao = freq_simulacao / len(simulacoes)

print("📊 Comparação: Teórico vs Simulação")
print("=" * 50)
print(f"{'k':<3} {'Teórica':<10} {'Simulação':<10} {'Diferença':<10}")
print("-" * 50)

for k in k_valores_2:
    diff = abs(prob_teoricas[k] - prob_simulacao[k])
    print(f"{k:<3} {prob_teoricas[k]:<10.4f} {prob_simulacao[k]:<10.4f} {diff:<10.4f}")

# Cálculo de propriedades
media_2 = n_pecas * p_defeito
variancia_2 = n_pecas * p_defeito * (1 - p_defeito)
desvio_padrao_2 = np.sqrt(variancia_2)

media_simulacao = np.mean(simulacoes)
desvio_simulacao = np.std(simulacoes)

print(f"\n📈 Propriedades Estatísticas:")
print(f"Média teórica: {media_2:.3f} | Média simulação: {media_simulacao:.3f}")
print(f"Desvio teórico: {desvio_padrao_2:.3f} | Desvio simulação: {desvio_simulacao:.3f}")

In [ ]:
# ENTÃO: Visualização detalhada dos resultados
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Gráfico 1: Comparação Teórico vs Simulação
x = np.arange(len(k_valores_2))
width = 0.35

bars1 = ax1.bar(x - width/2, prob_teoricas, width, label='Teórico', 
                color='lightcoral', alpha=0.8)
bars2 = ax1.bar(x + width/2, prob_simulacao, width, label='Simulação (1000 amostras)', 
                color='lightblue', alpha=0.8)

ax1.set_title(f'{nome_cenario_2}\nComparação: Teórico vs Simulação', fontweight='bold')
ax1.set_xlabel('Número de peças defeituosas (k)')
ax1.set_ylabel('Probabilidade')
ax1.set_xticks(x)
ax1.set_xticklabels(k_valores_2)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico 2: Histograma das simulações
ax2.hist(simulacoes, bins=np.arange(-0.5, n_pecas+1.5, 1), 
         density=True, alpha=0.7, color='green', edgecolor='black')
ax2.plot(k_valores_2, prob_teoricas, 'ro-', linewidth=2, markersize=6, 
         label='Distribuição teórica')
ax2.axvline(media_2, color='red', linestyle='--', linewidth=2, label=f'Média teórica = {media_2:.2f}')
ax2.axvline(media_simulacao, color='blue', linestyle='--', linewidth=2, label=f'Média simulação = {media_simulacao:.2f}')
ax2.set_title('Distribuição das Simulações', fontweight='bold')
ax2.set_xlabel('Número de defeitos por amostra')
ax2.set_ylabel('Densidade de probabilidade')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Gráfico 3: Análise de intervalos de confiança
intervalo_1_sigma = [max(0, media_2 - desvio_padrao_2), min(n_pecas, media_2 + desvio_padrao_2)]
intervalo_2_sigma = [max(0, media_2 - 2*desvio_padrao_2), min(n_pecas, media_2 + 2*desvio_padrao_2)]

ax3.bar(k_valores_2, prob_teoricas, color='lightgray', alpha=0.6, label='Distribuição completa')

# Destacando intervalos
mask_1sigma = (k_valores_2 >= intervalo_1_sigma[0]) & (k_valores_2 <= intervalo_1_sigma[1])
mask_2sigma = (k_valores_2 >= intervalo_2_sigma[0]) & (k_valores_2 <= intervalo_2_sigma[1])

ax3.bar(k_valores_2[mask_1sigma], prob_teoricas[mask_1sigma], 
        color='yellow', alpha=0.8, label='±1σ (68.3%)')
ax3.bar(k_valores_2[mask_2sigma], prob_teoricas[mask_2sigma], 
        color='orange', alpha=0.6, label='±2σ (95.4%)')

ax3.axvline(media_2, color='red', linestyle='-', linewidth=2, label=f'μ = {media_2:.2f}')
ax3.set_title('Intervalos de Confiança', fontweight='bold')
ax3.set_xlabel('Número de defeitos')
ax3.set_ylabel('Probabilidade')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Gráfico 4: Análise de risco e qualidade
prob_zero_defeitos = prob_teoricas[0]
prob_um_defeito = prob_teoricas[1]
prob_dois_ou_mais = sum(prob_teoricas[2:])
prob_acima_media = sum(prob_teoricas[k_valores_2 > media_2])

categorias = ['0 defeitos\n(Excelente)', '1 defeito\n(Bom)', '≥2 defeitos\n(Problemático)', 'Acima da média\n(Atenção)']
valores = [prob_zero_defeitos, prob_um_defeito, prob_dois_ou_mais, prob_acima_media]
cores = ['green', 'yellowgreen', 'orange', 'red']

bars = ax4.bar(categorias, valores, color=cores, alpha=0.8)
ax4.set_title('Análise de Risco da Qualidade', fontweight='bold')
ax4.set_ylabel('Probabilidade')
ax4.set_ylim(0, max(valores) * 1.1)

# Adicionando percentuais
for bar, valor in zip(bars, valores):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{valor*100:.1f}%', ha='center', va='bottom', fontweight='bold')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"\n🔍 Análise de Qualidade:")
print(f"• Probabilidade de amostra perfeita (0 defeitos): {prob_zero_defeitos*100:.2f}%")
print(f"• Probabilidade de amostra excelente (≤1 defeito): {(prob_zero_defeitos + prob_um_defeito)*100:.2f}%")
print(f"• Probabilidade de problemas (≥2 defeitos): {prob_dois_ou_mais*100:.2f}%")
print(f"• Número médio esperado de defeitos por amostra: {media_2:.2f}")

---
## 🧪 Cenário 3: Simulação Interativa - Experimento Personalizado

**DADO** que queremos experimentar com diferentes parâmetros  
**QUANDO** modificamos os valores de n e p  
**ENTÃO** podemos observar como a distribuição se comporta

In [ ]:
def experimento_binomial_interativo(n, p, titulo="Experimento Personalizado"):
    """
    Função para criar uma análise completa da distribuição binomial
    com parâmetros personalizados.
    """
    print(f"🔬 {titulo}")
    print(f"Parâmetros: n={n}, p={p}")
    print("=" * 50)
    
    # Calculando a distribuição
    k_valores = np.arange(0, n + 1)
    probabilidades = binom.pmf(k_valores, n, p)
    
    # Propriedades estatísticas
    media = n * p
    variancia = n * p * (1 - p)
    desvio_padrao = np.sqrt(variancia)
    moda = int(np.floor((n + 1) * p))
    
    # Simulação
    np.random.seed(42)
    simulacao = np.random.binomial(n, p, 10000)
    
    # Criando visualizações
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 10))
    
    # Gráfico 1: Distribuição principal
    bars = ax1.bar(k_valores, probabilidades, color='mediumseagreen', alpha=0.8, edgecolor='darkgreen')
    ax1.axvline(media, color='red', linestyle='--', linewidth=2, label=f'Média = {media:.2f}')
    ax1.axvline(moda, color='orange', linestyle='--', linewidth=2, label=f'Moda = {moda}')
    ax1.set_title(f'{titulo}\nDistribuição Binomial(n={n}, p={p})', fontweight='bold')
    ax1.set_xlabel('k (número de sucessos)')
    ax1.set_ylabel('P(X = k)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Destacando o valor mais provável
    max_prob_idx = np.argmax(probabilidades)
    bars[max_prob_idx].set_color('gold')
    bars[max_prob_idx].set_edgecolor('darkorange')
    bars[max_prob_idx].set_linewidth(2)
    
    # Gráfico 2: Simulação vs Teórico
    ax2.hist(simulacao, bins=np.arange(-0.5, n+1.5, 1), density=True, 
             alpha=0.6, color='lightblue', edgecolor='blue', label='Simulação (10,000)')
    ax2.plot(k_valores, probabilidades, 'ro-', markersize=6, linewidth=2, label='Teórico')
    ax2.set_title('Validação por Simulação', fontweight='bold')
    ax2.set_xlabel('k')
    ax2.set_ylabel('Densidade')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Gráfico 3: Função de distribuição acumulada
    cdf = binom.cdf(k_valores, n, p)
    ax3.step(k_valores, cdf, where='post', linewidth=3, color='purple')
    ax3.fill_between(k_valores, cdf, step='post', alpha=0.3, color='purple')
    ax3.set_title('Função de Distribuição Acumulada', fontweight='bold')
    ax3.set_xlabel('k')
    ax3.set_ylabel('P(X ≤ k)')
    ax3.grid(True, alpha=0.3)
    ax3.set_ylim(0, 1.05)
    
    # Gráfico 4: Análise de probabilidades especiais
    quartis = [
        ('P(X ≤ 25%)', binom.ppf(0.25, n, p)),
        ('P(X ≤ 50%)', binom.ppf(0.5, n, p)),
        ('P(X ≤ 75%)', binom.ppf(0.75, n, p)),
        ('Média', media),
        ('Moda', moda)
    ]
    
    labels, valores_quartis = zip(*quartis)
    cores_quartis = ['lightcoral', 'orange', 'lightgreen', 'red', 'gold']
    
    bars = ax4.barh(labels, valores_quartis, color=cores_quartis, alpha=0.8)
    ax4.set_title('Estatísticas Descritivas', fontweight='bold')
    ax4.set_xlabel('Valor de k')
    
    for bar, valor in zip(bars, valores_quartis):
        ax4.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2, 
                 f'{valor:.1f}', ha='left', va='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Relatório textual
    print(f"\n📊 Relatório de Análise:")
    print(f"• Média (μ): {media:.3f}")
    print(f"• Desvio Padrão (σ): {desvio_padrao:.3f}")
    print(f"• Variância (σ²): {variancia:.3f}")
    print(f"• Moda: {moda}")
    print(f"• Valor mais provável: k = {max_prob_idx} com P = {probabilidades[max_prob_idx]:.4f}")
    
    # Intervalos de interesse
    p_menor_media = sum(probabilidades[k_valores < media])
    p_maior_media = sum(probabilidades[k_valores > media])
    
    print(f"\n🎯 Probabilidades de Interesse:")
    print(f"• P(X < μ): {p_menor_media:.4f} ({p_menor_media*100:.2f}%)")
    print(f"• P(X > μ): {p_maior_media:.4f} ({p_maior_media*100:.2f}%)")
    
    # Verificação da lei dos grandes números
    media_simulacao = np.mean(simulacao)
    erro_simulacao = abs(media_simulacao - media)
    print(f"\n✅ Validação (Lei dos Grandes Números):")
    print(f"• Média teórica: {media:.4f}")
    print(f"• Média simulada: {media_simulacao:.4f}")
    print(f"• Erro absoluto: {erro_simulacao:.4f}")
    
    return {
        'parametros': {'n': n, 'p': p},
        'estatisticas': {'media': media, 'desvio_padrao': desvio_padrao, 'variancia': variancia, 'moda': moda},
        'probabilidades': probabilidades,
        'k_valores': k_valores,
        'simulacao': simulacao
    }

print("🎮 Função interativa criada! Use-a com diferentes parâmetros.")

In [ ]:
# EXEMPLO 1: Lançamento de moedas
resultado1 = experimento_binomial_interativo(n=10, p=0.5, titulo="Lançamento de 10 Moedas")

In [ ]:
# EXEMPLO 2: Testes de medicamento
resultado2 = experimento_binomial_interativo(n=20, p=0.8, titulo="Eficácia de Medicamento (20 pacientes)")

In [ ]:
# EXEMPLO 3: Evento raro
resultado3 = experimento_binomial_interativo(n=50, p=0.02, titulo="Eventos Raros (50 tentativas)")

---
## 📈 Análise Comparativa de Cenários

**DADO** que temos múltiplos cenários diferentes  
**QUANDO** comparamos suas características  
**ENTÃO** podemos entender melhor o comportamento da distribuição binomial

In [ ]:
# Definindo múltiplos cenários para comparação
cenarios_comparacao = [
    {'nome': 'Prova Difícil', 'n': 10, 'p': 0.2, 'cor': 'red'},
    {'nome': 'Prova Média', 'n': 10, 'p': 0.5, 'cor': 'blue'},
    {'nome': 'Prova Fácil', 'n': 10, 'p': 0.8, 'cor': 'green'},
    {'nome': 'Muitas Tentativas, p baixo', 'n': 50, 'p': 0.1, 'cor': 'purple'},
    {'nome': 'Poucas Tentativas, p alto', 'n': 5, 'p': 0.9, 'cor': 'orange'}
]

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

estatisticas_comparacao = []

for i, cenario in enumerate(cenarios_comparacao):
    n, p = cenario['n'], cenario['p']
    k_vals = np.arange(0, n + 1)
    probs = binom.pmf(k_vals, n, p)
    
    # Calculando estatísticas
    media = n * p
    variancia = n * p * (1 - p)
    desvio_padrao = np.sqrt(variancia)
    
    estatisticas_comparacao.append({
        'Cenário': cenario['nome'],
        'n': n,
        'p': p,
        'Média': media,
        'Desvio': desvio_padrao,
        'CV': desvio_padrao/media if media > 0 else 0  # Coeficiente de variação
    })
    
    # Plotando
    ax = axes[i]
    bars = ax.bar(k_vals, probs, color=cenario['cor'], alpha=0.7, edgecolor='black')
    ax.axvline(media, color='red', linestyle='--', linewidth=2, label=f'μ={media:.1f}')
    ax.set_title(f"{cenario['nome']}\n(n={n}, p={p})", fontweight='bold')
    ax.set_xlabel('k')
    ax.set_ylabel('P(X=k)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Destacar moda
    moda_idx = np.argmax(probs)
    bars[moda_idx].set_color('gold')
    bars[moda_idx].set_edgecolor('darkorange')
    bars[moda_idx].set_linewidth(2)

# Último gráfico: comparação das médias
ax = axes[5]
nomes = [c['nome'] for c in cenarios_comparacao]
medias = [c['n'] * c['p'] for c in cenarios_comparacao]
cores = [c['cor'] for c in cenarios_comparacao]

bars = ax.bar(range(len(nomes)), medias, color=cores, alpha=0.8)
ax.set_title('Comparação das Médias', fontweight='bold')
ax.set_ylabel('Média (μ = n×p)')
ax.set_xticks(range(len(nomes)))
ax.set_xticklabels(nomes, rotation=45, ha='right')

# Adicionando valores nas barras
for bar, media in zip(bars, medias):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             f'{media:.1f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Tabela comparativa
df_comparacao = pd.DataFrame(estatisticas_comparacao)
print("📊 Tabela Comparativa de Cenários:")
display(df_comparacao.round(3))

print("\n🔍 Observações:")
print("• Quanto maior 'p', mais a distribuição se desloca para a direita")
print("• Quanto maior 'n', mais a distribuição se aproxima da normal")
print("• A variabilidade é máxima quando p=0.5")
print("• O coeficiente de variação (CV) indica a dispersão relativa")

---
## 🎯 Exercícios Práticos para Experimentação

Use a função `experimento_binomial_interativo()` para explorar os seguintes cenários:

In [ ]:
print("🎮 EXERCÍCIOS PRÁTICOS:")
print("=" * 60)
print("\n1. 🎯 Marketing Digital:")
print("   Uma campanha tem 3% de taxa de conversão.")
print("   Se enviamos para 100 pessoas, quantas conversões esperamos?")
print("   Execute: experimento_binomial_interativo(100, 0.03, 'Campanha Marketing')")

print("\n2. 🏥 Teste Clínico:")
print("   Um novo tratamento tem 75% de eficácia.")
print("   Em um grupo de 15 pacientes, qual a probabilidade de diferentes resultados?")
print("   Execute: experimento_binomial_interativo(15, 0.75, 'Teste Clínico')")

print("\n3. 🎲 Análise de Jogos:")
print("   Em um jogo, você tem 1/6 de chance de ganhar cada rodada.")
print("   Jogando 30 vezes, quantas vitórias são esperadas?")
print("   Execute: experimento_binomial_interativo(30, 1/6, 'Jogo de Dados')")

print("\n4. 📊 Pesquisa de Opinião:")
print("   40% da população aprova uma medida.")
print("   Em uma amostra de 25 pessoas, como se distribui a aprovação?")
print("   Execute: experimento_binomial_interativo(25, 0.40, 'Pesquisa Opinião')")

print("\n5. 🔬 Experimento Livre:")
print("   Escolha seus próprios valores de n e p e observe os padrões!")
print("   Execute: experimento_binomial_interativo(SEU_N, SEU_P, 'Seu Título')")

print("\n" + "="*60)
print("💡 DICAS para exploração:")
print("• Teste com p muito pequeno (eventos raros)")
print("• Teste com p próximo de 0.5 (máxima variabilidade)")
print("• Teste com n muito grande (aproximação normal)")
print("• Compare cenários com mesmo μ = n×p mas diferentes n e p")

In [ ]:
# Espaço para seus experimentos
# Descomente e modifique as linhas abaixo para executar os exercícios:

# experimento_binomial_interativo(100, 0.03, 'Campanha Marketing')
# experimento_binomial_interativo(15, 0.75, 'Teste Clínico')
# experimento_binomial_interativo(30, 1/6, 'Jogo de Dados')
# experimento_binomial_interativo(25, 0.40, 'Pesquisa Opinião')
# experimento_binomial_interativo(SEU_N, SEU_P, 'Seu Título')

print("🔬 Remova os comentários (#) das linhas acima para executar os experimentos!")
print("✏️ Substitua 'SEU_N' e 'SEU_P' pelos valores que desejar testar.")

---
## 📝 Resumo e Conclusões

### ✅ O que aprendemos:

1. **Padrão BDD aplicado**: Estruturamos cada cenário com "Dado-Quando-Então" para maior clareza

2. **Cenários práticos**:
   - Provas de múltipla escolha (educação)
   - Controle de qualidade (indústria)
   - Diversos experimentos personalizáveis

3. **Visualizações importantes**:
   - Distribuição de probabilidades
   - Comparação teórico vs simulação
   - Função de distribuição acumulada
   - Análise de intervalos de confiança

4. **Propriedades fundamentais**:
   - Média: μ = n × p
   - Variância: σ² = n × p × (1-p)
   - Aproximação normal para n grande

### 🎯 Próximos passos:
- Explore diferentes combinações de n e p
- Compare com outras distribuições (Poisson, Normal)
- Aplique em problemas reais da sua área de interesse

### 📚 Para saber mais:
- Lei dos Grandes Números
- Teorema Central do Limite
- Distribuição de Poisson (casos especiais da Binomial)
